In [3]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os   
import pandas as pd
from sklearn.linear_model import LinearRegression
import math
import datetime 

years_to_average = [1961, 1990]
year = 2010

def regression(years, data):
    # Pass in panda series
    x = years.to_numpy()
    x = x.reshape((-1,1))
    y = data.to_numpy()
    #print('Performing Regression')
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x,y)
    intercept = model.intercept_
    coef = model.coef_
    return coef, intercept, r_sq


def plot_values(df_hist, df_projected, variable_name, yaxis_title, path):
    fig = go.Figure()

    ##### Historic Values #####
    fig.add_trace(go.Scatter(
        x = df_hist.index,
        y = df_hist[variable_name],
        name = 'Historic Observations ' + variable_name, 
        marker_color = 'darkgrey',
        marker_size = 6,
        mode = 'markers+lines'
    ))

    ##### RCP 4.5 #####
    fig.add_trace(go.Scatter(
        x = df_projected.index,
        y = df_projected[' rcp45_min'],
        marker_color = 'cornflowerblue',
        marker_size = 8,
        mode = 'lines',
        showlegend= False
    ))
    fig.add_trace(go.Scatter(
        x = df_projected.index,
        y = df_projected[' rcp45_weighted_mean'],
        name = 'RCP 4.5 ' + variable_name, 
        marker_color = 'cornflowerblue',
        marker_size = 16,
        mode = 'lines',
        fill='tonexty',
    ))
    fig.add_trace(go.Scatter(
        x = df_projected.index,
        y = df_projected[' rcp45_max'],
        marker_color = 'cornflowerblue',
        marker_size = 8,
        mode = 'lines',
        fill='tonexty',
        showlegend= False
    ))

    a, b, r_sw = regression(df_projected.index, df_projected[' rcp45_weighted_mean'])
    n = 100
    df_regression = pd.DataFrame(0.0, index=range(100), columns = ['x', 'y'])
    for i in df_regression.index:
        df_regression.loc[i,'x'] = min(df_projected.index) + i*(max(df_projected.index)-min(df_projected.index))/n
        df_regression.loc[i,'y'] = a*df_regression.loc[i,'x']+ b
    fig.add_trace(go.Scatter(
        x=df_regression['x'], 
        y=df_regression['y'],
        line=dict(color='cornflowerblue', width=2, dash = 'dash'),
        mode='lines',
        name='RCP 4.5 Regression'))


    ##### RCP 8.5 #####
    fig.add_trace(go.Scatter(
        x = df_projected.index,
        y = df_projected[' rcp85_min'],
        marker_color = 'indianred',
        marker_size = 8,
        mode = 'lines',
        showlegend= False
    ))
    fig.add_trace(go.Scatter(
        x = df_projected.index,
        y = df_projected[' rcp85_weighted_mean'],
        name = 'RCP 8.5 ' + variable_name, 
        marker_color = 'indianred',
        marker_size = 16,
        mode = 'lines',
        fill='tonexty'
    ))
    fig.add_trace(go.Scatter(
        x = df_projected.index,
        y = df_projected[' rcp85_max'],
        marker_color = 'indianred',
        marker_size = 8,
        mode = 'lines',
        fill='tonexty',
        showlegend= False
    ))

    a, b, r_sw = regression(df_projected.index, df_projected[' rcp85_weighted_mean'])
    n = 100
    df_regression = pd.DataFrame(0.0, index=range(100), columns = ['x', 'y'])
    for i in df_regression.index:
        df_regression.loc[i,'x'] = min(df_projected.index) + i*(max(df_projected.index)-min(df_projected.index))/n
        df_regression.loc[i,'y'] = a*df_regression.loc[i,'x']+ b
    fig.add_trace(go.Scatter(
        x=df_regression['x'], 
        y=df_regression['y'],
        line=dict(color='indianred', width=2, dash = 'dash'),
        mode='lines',
        name='RCP 8.5 Regression'))

    fig.update_xaxes(title = 'Year', range = [1960,2100])
    fig.update_yaxes(title = yaxis_title)
    if 'Cooling Degree Days' in variable_name:
        fig.update_yaxes(title = yaxis_title, range = [1000, 5500])
    fig.update_layout(template = 'simple_white', width = 1500, height = 500, font = dict(size = 16))
    fig.write_html(path+variable_name+'.html')
    fig.write_image(path+variable_name+'.png')
    fig.show()


In [4]:
path = 'Eddy_County/'
location = 'Eddy_County'

location_dict = {
    'Eddy_County': 'Eddy_County/',
    'Craighead_County': 'Craighead_County/',
    'Oklahoma_County': 'Oklahoma_County/',
    'Sequoyah_County': 'Sequoyah_County/',
    'Tulsa_County': 'Tulsa_County/',
    'Greene_County': 'Greene_County/'
}

for location in location_dict:
    print(location)
    path = location_dict[location]
    variable_dict = {
        # Cooling
        'Maximum T': {'label': ' tmax', 'h_obs': location+'-annual-hist_obs-tmax', 'h_mod': location+'-annual-hist_mod-tmax', 'p_mod': location+'-annual-proj_mod-tmax', 'y_axis': 'Maximum Daily Temperature (°F)'},
        'Cooling Degree Days': {'label': ' cdd_65f', 'h_obs': location+'-annual-hist_obs-cdd_65f', 'h_mod': location+'-annual-hist_mod-cdd_65f', 'p_mod': location+'-annual-proj_mod-cdd_65f', 'y_axis': 'Cooling Degree Days'},
        'Maximum GT 90': {'label': ' days_tmax_gt_90f', 'h_obs': location+'-annual-hist_obs-days_tmax_gt_90f', 'h_mod': location+'-annual-hist_mod-days_tmax_gt_90f', 'p_mod': location+'-annual-proj_mod-days_tmax_gt_90f', 'y_axis': 'Days w Maximum T > 90°F'},
        'Minimum GT 90': {'label': ' days_tmin_gt_90f', 'h_obs': location+'-annual-hist_obs-days_tmin_gt_90f', 'h_mod': location+'-annual-hist_mod-days_tmin_gt_90f', 'p_mod': location+'-annual-proj_mod-days_tmin_gt_90f', 'y_axis': 'Days w Minimum T > 90°F'},

        # Heating
        'Minimum T': {'label': ' tmin', 'h_obs': location+'-annual-hist_obs-tmin', 'h_mod': location+'-annual-hist_mod-tmin', 'p_mod': location+'-annual-proj_mod-tmin', 'y_axis': 'Minimum Daily Temperature (°F)'},
        'Heating Degree Days': {'label': ' hdd_65f', 'h_obs': location+'-annual-hist_obs-hdd_65f', 'h_mod': location+'-annual-hist_mod-hdd_65f', 'p_mod': location+'-annual-proj_mod-hdd_65f', 'y_axis': 'Heating Degree Days'},
        'Minimum LT 32': {'label': ' days_tmin_lt_32f', 'h_obs': location+'-annual-hist_obs-days_tmin_lt_32f', 'h_mod': location+'-annual-hist_mod-days_tmin_lt_32f', 'p_mod': location+'-annual-proj_mod-days_tmin_lt_32f', 'y_axis': 'Days w Minimum T < 32°F'},
        'Maximum LT 32': {'label': ' days_tmax_lt_32f', 'h_obs': location+'-annual-hist_obs-days_tmax_lt_32f', 'h_mod': location+'-annual-hist_mod-days_tmax_lt_32f', 'p_mod': location+'-annual-proj_mod-days_tmax_lt_32f', 'y_axis': 'Days w Maximum T < 32°F'},
        

        # Precipitation
        'Annual Precipitation': {'label': ' pcpn', 'h_obs': location+'-annual-hist_obs-pcpn', 'h_mod': location+'-annual-hist_mod-pcpn', 'p_mod': location+'-annual-proj_mod-pcpn', 'y_axis': 'Annual Precipitation'},
        'Dry Days': {'label': ' days_dry_days', 'h_obs': location+'-annual-hist_obs-days_dry_days', 'h_mod': location+'-annual-hist_mod-days_dry_days', 'p_mod': location+'-annual-proj_mod-days_dry_days', 'y_axis': 'Dry Days'},
        'Wet Days': {'label': ' days_pcpn_gt_2in', 'h_obs': location+'-annual-hist_obs-days_pcpn_gt_2in', 'h_mod': location+'-annual-hist_mod-days_pcpn_gt_2in', 'p_mod': location+'-annual-proj_mod-days_pcpn_gt_2in', 'y_axis': 'High Precipitation Days'},
    }

    for variable in variable_dict:
        historical_observed = variable_dict[variable]['h_obs']
        historical_modeled = variable_dict[variable]['h_mod']
        projected_modeled = variable_dict[variable]['p_mod']

        df_obs = pd.read_csv(path+historical_observed+'.csv', index_col = 'year')
        df_modeled = pd.read_csv(path+projected_modeled+'.csv', index_col = 'year')

        print(str(years_to_average), variable_dict[variable]['label'], round(df_obs.loc[years_to_average[0]:years_to_average[1], variable_dict[variable]['label']].mean(),2))
        print(str(year), variable_dict[variable]['label'], df_obs.loc[year, variable_dict[variable]['label']])

        plot_values(df_obs, df_modeled, variable_dict[variable]['label'], variable_dict[variable]['y_axis'], path)

        



Eddy_County
[1961, 1990]  tmax 75.72
2010  tmax 76.5


[1961, 1990]  cdd_65f 1666.67
2010  cdd_65f 1800


[1961, 1990]  days_tmax_gt_90f 80.49
2010  days_tmax_gt_90f 90.0


[1961, 1990]  days_tmin_gt_90f 0.0
2010  days_tmin_gt_90f 0


[1961, 1990]  tmin 45.93
2010  tmin 46.5


...

[1961, 1990]  days_tmin_lt_32f 100.17
2010  days_tmin_lt_32f 98.6


[1961, 1990]  days_tmax_lt_32f 16.37
2010  days_tmax_lt_32f 19.8


[1961, 1990]  pcpn 42.69
2010  pcpn 48.83


[1961, 1990]  days_dry_days 212.06
2010  days_dry_days 191.3


[1961, 1990]  days_pcpn_gt_2in 1.02
2010  days_pcpn_gt_2in 1.5
